# Problema de negócio
Coletar dados de livros no site  https://books.toscrape.com

1. Saída: ( Produto final )

1.1. A resposta para a pergunta.

    Fornecer uma tabela com os dados coletados
    
1.2. Formato da entrega
    
    Tabela
    
1.3. Local da entrega

    csv com todas as informações solicitadas

2. Processo ( Passo a Passo )

2.1. Passo a passso para construir a tabela

    Contruir a tabela em um dataframe

2.2. Definir o formato da entrega ( Tabela)

Coletar dados so site:
- Tabela com as seguintes colunas
    id	Catálogo	Nome do livro	Preço 	Avaliação 	Disponibilidade	dataScraping
    
    - Definição do schema: Colunas e seu tipo
    
    
- Entrega do produto final
3. Decidir o local de entrega

    - Tabela csv



4. Coletar os seguintes dados da página:

• Catálogo:

– Classics
– Science Fiction
– Humor
– Business

• Coletar os seguintes dados de cada livro:

– Nome do livro
– Preço em libras
– Avaliação dos consumidores
– Disponível em estoque


1. Coletar os seguintes dados da página: https://books.toscrape.com

    • Catálogo:
   
   – Classics
    
    – Science Fiction
    
    – Humor
    
    – Business
    
    • Coletar os seguintes dados de cada livro:
    
    – Nome do livro
    
    – Preço em libras
    
    – Avaliação dos consumidores
    
    – Disponível em estoque

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Categória Classics
lista=['classics_6','science-fiction_16', 'humor_30', 'business_35']

#empty dataframe
data_livros=pd.DataFrame()

for i in lista:

    url='https://books.toscrape.com/catalogue/category/books/'+i+'/index.html'


    page = requests.get( url, headers=headers )

    soup=BeautifulSoup( page.text, 'html.parser' )
    livros=soup.find('ol', class_="row")


    #================== nome dos livros===========================
    livro_list=livros.find_all('article', class_="product_pod")
    livro_name=[p.find_all('a')[1].get('title') for p in livro_list]

    # ====================Preços dos livros=====================
    precos=livros.find_all('p', class_="price_color" )
    preco_livros=[p.get_text() for p in precos]
    precos_livros_numerico=[p[2:] for p in preco_livros]

    # ===================Item em estoque========================
    Estoque=livros.find_all('p',class_="instock availability" )
    Item_Estoque=[list(filter(None, p.get_text().split('\n')))[1] for p in Estoque]

    # ===================Estrelas=============================
    stars=livros.find_all('article',class_="product_pod" )
    stars[1].find_all('p')[0]
    star=[p.find_all('p')[0] for p in stars]
    

    #===================Adicionando o catálogo==================    
    catalogo=[]
    for j in range(len(livro_name)):
        catalogo.append(i)
 
    #====================dataFrame==============================
    data_livros=data_livros.append(pd.DataFrame([livro_name, Item_Estoque,precos_livros_numerico,catalogo, star]).T,ignore_index=True )
        
    
data_livros.columns=['name_livro','Estoque','valor_libras','catalogo', 'star']

# Aplicar regex para pegar apenas a quantidade de estrelas
# ============mudar para string===========================
data_livros['star']=data_livros['star'].apply(str)
#=============aplicando regex=============================
data_livros['star']=data_livros['star'].apply(lambda x : re.search('([A-Z]\w+)', x ).group(1) if pd.notnull(x) else x ) 

#================ Utilizando o regex no catálogo====================
data_livros['catalogo']=data_livros['catalogo'].apply(lambda x : re.search('(\w+.\w+)_\d+', x ).group(1) if pd.notnull(x) else x )


In [3]:
data_livros.head()

,name_livro,Estoque,valor_libras,catalogo,star
0,The Secret Garden,In stock,15.08,classics,Four
1,The Metamorphosis,In stock,28.58,classics,One
2,The Pilgrim's Progress,In stock,50.26,classics,Two
3,The Hound of the Baskervilles (Sherlock Holmes...,In stock,14.82,classics,Two
4,Little Women (Little Women #1),In stock,28.07,classics,Four


In [4]:
data_livros.to_csv('books.csv', index=False)

In [5]:
data_livros.apply(lambda x: len(x.unique()))

name_livro      57
Estoque          1
valor_libras    57
catalogo         4
star             5
dtype: int64

In [6]:
# Drop duplicates
data_livros=data_livros.drop_duplicates()

In [7]:
data_livros

,name_livro,Estoque,valor_libras,catalogo,star
0,The Secret Garden,In stock,15.08,classics,Four
1,The Metamorphosis,In stock,28.58,classics,One
2,The Pilgrim's Progress,In stock,50.26,classics,Two
3,The Hound of the Baskervilles (Sherlock Holmes...,In stock,14.82,classics,Two
4,Little Women (Little Women #1),In stock,28.07,classics,Four
5,Gone with the Wind,In stock,32.49,classics,Three
6,Candide,In stock,58.63,classics,Three
7,Animal Farm,In stock,57.22,classics,Three
8,Wuthering Heights,In stock,17.73,classics,Three
9,The Picture of Dorian Gray,In stock,29.70,classics,Two
